## Recommendations with MovieTweetings: Collaborative Filtering

One of the most popular methods for making recommendations is **collaborative filtering**.  In collaborative filtering, you are using the collaboration of user-item recommendations to assist in making new recommendations.  

There are two main methods of performing collaborative filtering:

1. **Neighborhood-Based Collaborative Filtering**, which is based on the idea that we can either correlate items that are similar to provide recommendations or we can correlate users to one another to provide recommendations.

2. **Model Based Collaborative Filtering**, which is based on the idea that we can use machine learning and other mathematical models to understand the relationships that exist amongst items and users to predict ratings and provide ratings.


In this notebook, you will be working on performing **neighborhood-based collaborative filtering**.  There are two main methods for performing collaborative filtering:

1. **User-based collaborative filtering:** In this type of recommendation, users related to the user you would like to make recommendations for are used to create a recommendation.

2. **Item-based collaborative filtering:** In this type of recommendation, first you need to find the items that are most related to each other item (based on similar ratings).  Then you can use the ratings of an individual on those similar items to understand if a user will like the new item.

In this notebook you will be implementing **user-based collaborative filtering**.  However, it is easy to extend this approach to make recommendations using **item-based collaborative filtering**.  First, let's read in our data and necessary libraries.

**NOTE**: Because of the size of the datasets, some of your code cells here will take a while to execute, so be patient!

In [1]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import tests as t
from scipy.sparse import csr_matrix
from IPython.display import HTML


%matplotlib inline

# Read in the datasets
movies = pd.read_csv('movies_clean.csv')
reviews = pd.read_csv('reviews_clean.csv')

del movies['Unnamed: 0']
del reviews['Unnamed: 0']

print(reviews.head())

   user_id  movie_id  rating   timestamp                 date  month_1  \
0        1     68646      10  1381620027  2013-10-12 23:20:27        0   
1        1    113277      10  1379466669  2013-09-18 01:11:09        0   
2        2    422720       8  1412178746  2014-10-01 15:52:26        0   
3        2    454876       8  1394818630  2014-03-14 17:37:10        0   
4        2    790636       7  1389963947  2014-01-17 13:05:47        0   

   month_2  month_3  month_4  month_5  ...  month_9  month_10  month_11  \
0        0        0        0        0  ...        0         1         0   
1        0        0        0        0  ...        0         0         0   
2        0        0        0        0  ...        0         1         0   
3        0        0        0        0  ...        0         0         0   
4        0        0        0        0  ...        0         0         0   

   month_12  year_2013  year_2014  year_2015  year_2016  year_2017  year_2018  
0         0          1  

In [2]:
movies.head()

,movie_id,movie,genre,date,1800's,1900's,2000's,History,News,Horror,...,Fantasy,Romance,Game-Show,Action,Documentary,Animation,Comedy,Short,Western,Thriller
0,8,Edison Kinetoscopic Record of a Sneeze (1894),Documentary|Short,1894,1,0,0,0,0,0,...,0,0,0,0,1,0,0,1,0,0
1,10,La sortie des usines Lumière (1895),Documentary|Short,1895,1,0,0,0,0,0,...,0,0,0,0,1,0,0,1,0,0
2,12,The Arrival of a Train (1896),Documentary|Short,1896,1,0,0,0,0,0,...,0,0,0,0,1,0,0,1,0,0
3,25,The Oxford and Cambridge University Boat Race ...,NaN,1895,1,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
4,91,Le manoir du diable (1896),Short|Horror,1896,1,0,0,0,0,1,...,0,0,0,0,0,0,0,1,0,0


### Measures of Similarity

When using **neighborhood** based collaborative filtering, it is important to understand how to measure the similarity of users or items to one another.  

There are a number of ways in which we might measure the similarity between two vectors (which might be two users or two items).  In this notebook, we will look specifically at two measures used to compare vectors:

* **Pearson's correlation coefficient**

Pearson's correlation coefficient is a measure of the strength and direction of a linear relationship. The value for this coefficient is a value between -1 and 1 where -1 indicates a strong, negative linear relationship and 1 indicates a strong, positive linear relationship. 

If we have two vectors x and y, we can define the correlation between the vectors as:


$$CORR(x, y) = \frac{\text{COV}(x, y)}{\text{STDEV}(x)\text{ }\text{STDEV}(y)}$$

where 

$$\text{STDEV}(x) = \sqrt{\frac{1}{n-1}\sum_{i=1}^{n}(x_i - \bar{x})^2}$$

and 

$$\text{COV}(x, y) = \frac{1}{n-1}\sum_{i=1}^{n}(x_i - \bar{x})(y_i - \bar{y})$$

where n is the length of the vector, which must be the same for both x and y and $\bar{x}$ is the mean of the observations in the vector.  

We can use the correlation coefficient to indicate how alike two vectors are to one another, where the closer to 1 the coefficient, the more alike the vectors are to one another.  There are some potential downsides to using this metric as a measure of similarity.  You will see some of these throughout this workbook.


* **Euclidean distance**

Euclidean distance is a measure of the straightline distance from one vector to another.  Because this is a measure of distance, larger values are an indication that two vectors are different from one another (which is different than Pearson's correlation coefficient).

Specifically, the euclidean distance between two vectors x and y is measured as:

$$ \text{EUCL}(x, y) = \sqrt{\sum_{i=1}^{n}(x_i - y_i)^2}$$

Different from the correlation coefficient, no scaling is performed in the denominator.  Therefore, you need to make sure all of your data are on the same scale when using this metric.

**Note:** Because measuring similarity is often based on looking at the distance between vectors, it is important in these cases to scale your data or to have all data be in the same scale.  If some measures are on a 5 point scale, while others are on a 100 point scale, you are likely to have non-optimal results due to the difference in variability of your features.  In this case, we will not need to scale data because they are all on a 10 point scale, but it is always something to keep in mind!

------------

### User-Item Matrix

In order to calculate the similarities, it is common to put values in a matrix.  In this matrix, users are identified by each row, and items are represented by columns.  


![alt text](imgs/userxitem.png "User Item Matrix")


In the above matrix, you can see that **User 1** and **User 2** both used **Item 1**, and **User 2**, **User 3**, and **User 4** all used **Item 2**.  However, there are also a large number of missing values in the matrix for users who haven't used a particular item.  A matrix with many missing values (like the one above) is considered **sparse**.

Our first goal for this notebook is to create the above matrix with the **reviews** dataset.  However, instead of 1 values in each cell, you should have the actual rating.  

The users will indicate the rows, and the movies will exist across the columns. To create the user-item matrix, we only need the first three columns of the **reviews** dataframe, which you can see by running the cell below.

In [53]:
user_items = reviews[['user_id', 'movie_id', 'rating']]
user_items = user_items.sample(n=10000,random_state=42)

In [54]:
user_items.user_id.unique()

array([23915, 11830, 38147, ..., 28952, 23528, 18307], dtype=int64)

### Creating the User-Item Matrix

In order to create the user-items matrix (like the one above), I personally started by using a [pivot table](https://pandas.pydata.org/pandas-docs/stable/generated/pandas.pivot_table.html). 

However, I quickly ran into a memory error (a common theme throughout this notebook).  I will help you navigate around many of the errors I had, and achieve useful collaborative filtering results! 

_____

`1.` Create a matrix where the users are the rows, the movies are the columns, and the ratings exist in each cell, or a NaN exists in cells where a user hasn't rated a particular movie. If you get a memory error (like I did), [this link here](https://stackoverflow.com/questions/39648991/pandas-dataframe-pivot-memory-error) might help you!

In [55]:
user_items.user_id = user_items.user_id.astype('uint16')
user_items.movie_id = user_items.movie_id.astype('uint16')
user_items.rating = user_items.rating.astype('uint8')

In [56]:
# Create user-by-item matrix
user_by_movie = user_items.groupby(['user_id', 'movie_id'])['rating'].max().unstack()

In [57]:
user_by_movie.head()

movie_id,10,35,44,78,120,124,146,148,153,159,...,65293,65309,65410,65416,65421,65442,65448,65464,65491,65500
user_id,,,,,,,,,,,,,,,,,,,,,
17,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
20,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
31,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
32,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
33,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


Check your results below to make sure your matrix is ready for the upcoming sections.

In [58]:
assert movies.shape[0] == user_by_movie.shape[1], "Oh no! Your matrix should have {} columns, and yours has {}!".format(movies.shape[0], user_by_movie.shape[1])
assert reviews.user_id.nunique() == user_by_movie.shape[0], "Oh no! Your matrix should have {} rows, and yours has {}!".format(reviews.user_id.nunique(), user_by_movie.shape[0])
print("Looks like you are all set! Proceed!")
HTML('<img src="images/greatjob.webp">')

AssertionError: Oh no! Your matrix should have 31245 columns, and yours has 3641!

`2.` Now that you have a matrix of users by movies, use this matrix to create a dictionary where the key is each user and the value is an array of the movies each user has rated.

In [59]:
# Create a dictionary with users and corresponding movies seen

def movies_watched(user_id):
    '''
    INPUT:
    user_id - the user_id of an individual as int
    OUTPUT:
    movies - an array of movies the user has watched
    '''
    movie = user_by_movie.loc[user_id].dropna().index.values

    return movie


def create_user_movie_dict():
    '''
    INPUT: None
    OUTPUT: movies_seen - a dictionary where each key is a user_id and the value is an array of movie_ids
    
    Creates the movies_seen dictionary
    '''
    movies_seen = dict()
    for user in user_by_movie.index.values:
        movies_seen[user] = movies_watched(user)
    
    
    return movies_seen


# Use your function to return dictionary
movies_seen = create_user_movie_dict()

In [60]:
movies_seen

{17: array([34138], dtype=uint64),
 20: array([32919], dtype=uint64),
 31: array([34244], dtype=uint64),
 32: array([63962], dtype=uint64),
 33: array([49632, 60920], dtype=uint64),
 44: array([60804], dtype=uint64),
 51: array([44937, 51658], dtype=uint64),
 55: array([29212], dtype=uint64),
 66: array([45422, 50200, 55353], dtype=uint64),
 74: array([3110], dtype=uint64),
 90: array([9692], dtype=uint64),
 104: array([689], dtype=uint64),
 112: array([21337, 57866], dtype=uint64),
 119: array([25349, 53598], dtype=uint64),
 122: array([ 2311, 19869], dtype=uint64),
 125: array([12432], dtype=uint64),
 155: array([22380], dtype=uint64),
 157: array([4497], dtype=uint64),
 159: array([17042, 57810, 58464], dtype=uint64),
 162: array([25480], dtype=uint64),
 164: array([44066, 59200], dtype=uint64),
 171: array([35116], dtype=uint64),
 177: array([6491], dtype=uint64),
 212: array([53604], dtype=uint64),
 214: array([41930], dtype=uint64),
 219: array([12976], dtype=uint64),
 256: array

`3.` If a user hasn't rated more than 2 movies, we consider these users "too new".  Create a new dictionary that only contains users who have rated more than 2 movies.  This dictionary will be used for all the final steps of this workbook.

In [61]:
# Remove individuals who have watched 2 or fewer movies - don't have enough data to make recs

def create_movies_to_analyze(movies_seen, lower_bound=2):
    '''
    INPUT:  
    movies_seen - a dictionary where each key is a user_id and the value is an array of movie_ids
    lower_bound - (an int) a user must have more movies seen than the lower bound to be added to the movies_to_analyze dictionary

    OUTPUT: 
    movies_to_analyze - a dictionary where each key is a user_id and the value is an array of movie_ids
    
    The movies_seen and movies_to_analyze dictionaries should be the same except that the output dictionary has removed 
    
    '''
    
    # Do things to create updated dictionary
    movies_to_analyze = dict()
    for k in movies_seen.keys():
        if len(movies_seen[k])>lower_bound:
            movies_to_analyze[k] = movies_seen[k]
    
    return movies_to_analyze


# Use your function to return your updated dictionary
movies_to_analyze = create_movies_to_analyze(movies_seen)

In [62]:
len(movies_to_analyze.keys())

856

In [63]:
movies_to_analyze

{66: array([45422, 50200, 55353], dtype=uint64),
 159: array([17042, 57810, 58464], dtype=uint64),
 443: array([ 7705, 21468, 47999, 48833], dtype=uint64),
 471: array([42652, 52079, 55836], dtype=uint64),
 472: array([42824, 46730, 60976], dtype=uint64),
 715: array([ 9883, 11291, 16809, 34949], dtype=uint64),
 826: array([ 9692, 30260, 39832], dtype=uint64),
 882: array([10806, 29574, 39641], dtype=uint64),
 903: array([24807, 33514, 40160], dtype=uint64),
 992: array([ 4016, 18720, 19398, 28114, 32860, 34244, 41608, 49807, 62618,
        65057], dtype=uint64),
 1047: array([26942, 36670, 41212, 54941], dtype=uint64),
 1093: array([ 7354, 32401, 52077, 54670, 58389, 64115, 64665], dtype=uint64),
 1189: array([34994, 35133, 39832], dtype=uint64),
 1196: array([19684, 45966, 52796], dtype=uint64),
 1209: array([ 1358,  3064,  4142, 28681, 33890, 43662, 50717, 64326],
       dtype=uint64),
 1259: array([ 6026,  6411, 35284], dtype=uint64),
 1264: array([49642, 58547, 60445], dtype=uint6

In [64]:
# Run the tests below to check that your movies_to_analyze matches the solution
assert len(movies_to_analyze) == 23512, "Oops!  It doesn't look like your dictionary has the right number of individuals."
assert len(movies_to_analyze[2]) == 23, "Oops!  User 2 didn't match the number of movies we thought they would have."
assert len(movies_to_analyze[7])  == 3, "Oops!  User 7 didn't match the number of movies we thought they would have."
print("If this is all you see, you are good to go!")

AssertionError: Oops!  It doesn't look like your dictionary has the right number of individuals.

### Calculating User Similarities

Now that you have set up the **movies_to_analyze** dictionary, it is time to take a closer look at the similarities between users.  Below is the pseudocode for how I thought about determining the similarity between users:

```
for user1 in movies_to_analyze
    for user2 in movies_to_analyze
        see how many movies match between the two users
        if more than two movies in common
            pull the overlapping movies
            compute the distance/similarity metric between ratings on the same movies for the two users
            store the users and the distance metric
```

However, this took a very long time to run, and other methods of performing these operations did not fit on the workspace memory!

Therefore, rather than creating a dataframe with all possible pairings of users in our data, your task for this question is to look at a few specific examples of the correlation between ratings given by two users.  For this question consider you want to compute the [correlation](https://pandas.pydata.org/pandas-docs/stable/generated/pandas.DataFrame.corr.html) between users.

`4.` Using the **movies_to_analyze** dictionary and **user_by_movie** dataframe, create a function that computes the correlation between the ratings of similar movies for two users.  Then use your function to compare your results to ours using the tests below.  

In [65]:
def compute_correlation(user1, user2):
    '''
    INPUT
    user1 - int user_id
    user2 - int user_id
    OUTPUT
    the correlation between the matching ratings between the two users
    '''
    m1 = movies_to_analyze[user1]
    m2 = movies_to_analyze[user2]
    
    common = np.intersect1d(m1,m2,assume_unique=True)
    
    corr = user_by_movie.loc[[user1,user2],common].T.corr().iloc[0,1]
    
    return corr #return the correlation

In [66]:
# Test your function against the solution
assert compute_correlation(2,2) == 1.0, "Oops!  The correlation between a user and itself should be 1.0."
assert round(compute_correlation(2,66), 2) == 0.76, "Oops!  The correlation between user 2 and 66 should be about 0.76."
assert np.isnan(compute_correlation(2,104)), "Oops!  The correlation between user 2 and 104 should be a NaN."

print("If this is all you see, then it looks like your function passed all of our tests!")

KeyError: 2

### Why the NaN's?

If the function you wrote passed all of the tests, then you have correctly set up your function to calculate the correlation between any two users.  

`5.` But one question is, why are we still obtaining **NaN** values?  As you can see in the code cell above, users 2 and 104 have a correlation of **NaN**. Why?

Think and write your ideas here about why these NaNs exist, and use the cells below to do some coding to validate your thoughts. You can check other pairs of users and see that there are actually many NaNs in our data - 2,526,710 of them in fact. These NaN's ultimately make the correlation coefficient a less than optimal measure of similarity between two users.




In [68]:
# Which movies did both user 2 and user 104 see?
compute_correlation(66,39788)

nan

In [67]:
# What were the ratings for each user for those movies?
m1 = movies_to_analyze[66]
m2 = movies_to_analyze[39788]

common = np.intersect1d(m1,m2,assume_unique=True)
user_by_movie.loc[[66,39788],common]

movie_id,50200
user_id,
66,8.0
39788,8.0


`6.` Because the correlation coefficient proved to be less than optimal for relating user ratings to one another, we could instead calculate the euclidean distance between the ratings.  I found [this post](https://stackoverflow.com/questions/1401712/how-can-the-euclidean-distance-be-calculated-with-numpy) particularly helpful when I was setting up my function.  This function should be very similar to your previous function.  When you feel confident with your function, test it against our results.

In [69]:
def compute_euclidean_dist(user1, user2):
    '''
    INPUT
    user1 - int user_id
    user2 - int user_id
    OUTPUT
    the euclidean distance between user1 and user2
    '''
    m1 = movies_to_analyze[user1]
    m2 = movies_to_analyze[user2]
    
    common = np.intersect1d(m1,m2,assume_unique=True)
    
    d = user_by_movie.loc[[user1,user2],common]
    dist = np.linalg.norm(d.loc[user1]-d.loc[user2])
    
   
    return dist #return the euclidean distance

In [70]:
compute_euclidean_dist(66,39788)

0.0

In [71]:
# Test your function against the solution
assert compute_euclidean_dist(2,2) == 0, "Oops! Wrong"
assert round(compute_euclidean_dist(54,44), 2) == 5.1, "Oops! Wrong"
assert np.isnan(compute_euclidean_dist(2,104)) == False, "Oops! Wrong"

print("If this is all you see, then it looks like your function passed all of our tests!")

KeyError: 2

In [72]:
# Read in solution euclidean distances"
import pickle
df_dists = pd.read_pickle("dists.p")

In [73]:
df_dists.head()

,user1,user2,eucl_dist
0,2,2,0.000000
1,2,66,2.236068
2,2,90,5.385165
3,2,99,2.828427
4,2,104,2.000000


In [74]:
df_dists.query('user1 == 2').sort_values(by='eucl_dist').iloc[1:]['user2']

35        755
1161    22915
1836    35310
1808    34706
1729    33207
        ...  
2575    49739
165      3514
1915    36807
1699    32494
2743    52737
Name: user2, Length: 2808, dtype: int64

### Using the Nearest Neighbors to Make Recommendations

In the previous question, you read in **df_dists**. Therefore, you have a measure of distance between each user and every other user. This dataframe holds every possible pairing of users, as well as the corresponding euclidean distance.

Because of the **NaN** values that exist within the correlations of the matching ratings for many pairs of users, as we discussed above, we will proceed using **df_dists**. You will want to find the users that are 'nearest' each user.  Then you will want to find the movies the closest neighbors have liked to recommend to each user.

I made use of the following objects:

* df_dists (to obtain the neighbors)
* user_items (to obtain the movies the neighbors and users have rated)
* movies (to obtain the names of the movies)

`7.` Complete the functions below, which allow you to find the recommendations for any user.  There are five functions which you will need:

* **find_closest_neighbors** - this returns a list of user_ids from closest neighbor to farthest neighbor using euclidean distance


* **movies_liked** - returns an array of movie_ids


* **movie_names** - takes the output of movies_liked and returns a list of movie names associated with the movie_ids


* **make_recommendations** - takes a user id and goes through closest neighbors to return a list of movie names as recommendations


* **all_recommendations** = loops through every user and returns a dictionary of with the key as a user_id and the value as a list of movie recommendations

In [75]:
user_by_movie.head()

movie_id,10,35,44,78,120,124,146,148,153,159,...,65293,65309,65410,65416,65421,65442,65448,65464,65491,65500
user_id,,,,,,,,,,,,,,,,,,,,,
17,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
20,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
31,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
32,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
33,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [77]:
def find_closest_neighbors(user):
    '''
    INPUT:
        user - (int) the user_id of the individual you want to find the closest users
    OUTPUT:
        closest_neighbors - an array of the id's of the users sorted from closest to farthest away
    '''
    # I treated ties as arbitrary and just kept whichever was easiest to keep using the head method
    # You might choose to do something less hand wavy - order the neighbors
    closest = np.array(df_dists.query(f'user1 == {user}').sort_values(by='eucl_dist').iloc[1:]['user2'])
    
    
    return closest
    
    
    
def movies_liked(user_id, min_rating=7):
    '''
    INPUT:
    user_id - the user_id of an individual as int
    min_rating - the minimum rating considered while still a movie is still a "like" and not a "dislike"
    OUTPUT:
    movies_liked - an array of movies the user has watched and liked
    '''
    movies_liked = user_items.query(f'user_id == {user_id} and rating > {min_rating}')['movie_id'].values
  
    return movies_liked


def movie_names(movie_ids):
    '''
    INPUT
    movie_ids - a list of movie_ids
    OUTPUT
    movies - a list of movie names associated with the movie_ids
    
    '''
    movie_lst = list(movies[movies.movie_id.isin(movie_ids)]['movie'])
    return movie_lst
    
    
def make_recommendations(user, num_recs=10):
    '''
    INPUT:
        user - (int) a user_id of the individual you want to make recommendations for
        num_recs - (int) number of movies to return
    OUTPUT:
        recommendations - a list of movies - if there are "num_recs" recommendations return this many
                          otherwise return the total number of recommendations available for the "user"
                          which may just be an empty list
    '''
    movies_seen = movies_watched(user)
    neighbors = find_closest_neighbors(user)
    
    recs_id = []
    for i in neighbors:
        liked = movies_liked(i)
        tmp = np.setdiff1d(liked,movies_seen,assume_unique=True)
        recs = np.setdiff1d(tmp,np.array(recs_id))
        recs_id = recs_id + recs.tolist()
        if len(recs_id) >= num_recs:
            break
    recommendations = movie_names(recs_id[:10])        

    
    return recommendations

def all_recommendations(num_recs=10):
    '''
    INPUT 
        num_recs (int) the (max) number of recommendations for each user
    OUTPUT
        all_recs - a dictionary where each key is a user_id and the value is an array of recommended movie titles
    '''
    

    # Make the recommendations for each user
    all_recs = dict()
    user_ids = movies_to_analyze.keys()
    
    for u in user_ids:
        all_recs[u] = make_recommendations(u, num_recs)
    
    return all_recs

all_recs = all_recommendations(10)

In [80]:
all_recs.keys()

dict_keys([66, 159, 443, 471, 472, 715, 826, 882, 903, 992, 1047, 1093, 1189, 1196, 1209, 1259, 1264, 1413, 1609, 1652, 1677, 1853, 2127, 2175, 2317, 2365, 2493, 2548, 2636, 2681, 2692, 2701, 2704, 2771, 2803, 2828, 2917, 3052, 3064, 3317, 3380, 3397, 3459, 3499, 3514, 3532, 3679, 3728, 3739, 3754, 3779, 3840, 3843, 3844, 3880, 3892, 3934, 4029, 4035, 4065, 4186, 4187, 4225, 4259, 4268, 4536, 4567, 4642, 4693, 4694, 4703, 4714, 4857, 4872, 4900, 4996, 5043, 5047, 5084, 5103, 5359, 5440, 5495, 5508, 5545, 5549, 5597, 5776, 5806, 5901, 6061, 6094, 6260, 6338, 6360, 6431, 6453, 6458, 6481, 6505, 6550, 6571, 6639, 6682, 6742, 6792, 6855, 6936, 6937, 7093, 7122, 7151, 7172, 7302, 7498, 7619, 7687, 7741, 7863, 7906, 8074, 8116, 8138, 8211, 8266, 8310, 8408, 8482, 8648, 8711, 8735, 8796, 8867, 8932, 8974, 9024, 9109, 9119, 9198, 9317, 9349, 9356, 9373, 9437, 9474, 9551, 9687, 9720, 9755, 9762, 9864, 9882, 9911, 9914, 9970, 10076, 10138, 10154, 10182, 10444, 10481, 10489, 10741, 10761, 10790, 

In [78]:
# This loads our solution dictionary so you can compare results
all_recs_sol = pd.read_pickle("all_recs.p")

In [81]:
assert all_recs[66] == make_recommendations(66), "Oops!  Your recommendations for user 32 didn't match ours."
assert all_recs[39788] == make_recommendations(39788), "Oops!  It actually wasn't possible to make any recommendations for user 26."
assert all_recs[159] == make_recommendations(159), "Oops! Looks like your solution for user 1632 didn't match ours."
print("If you made it here, you now have recommendations for many users using collaborative filtering!")
HTML('<img src="images/greatjob.webp">')

If you made it here, you now have recommendations for many users using collaborative filtering!


### Now What?

If you made it this far, you have successfully implemented a solution to making recommendations using collaborative filtering. 

`8.` Let's do a quick recap of the steps taken to obtain recommendations using collaborative filtering.  

In [83]:
# Check your understanding of the results by correctly filling in the dictionary below
a = "pearson's correlation and spearman's correlation"
b = 'item based collaborative filtering'
c = "there were too many ratings to get a stable metric"
d = 'user based collaborative filtering'
e = "euclidean distance and pearson's correlation coefficient"
f = "manhattan distance and euclidean distance"
g = "spearman's correlation and euclidean distance"
h = "the spread in some ratings was zero"
i = 'content based recommendation'

sol_dict = {
    'The type of recommendation system implemented here was a ...': d,# letter here,
    'The two methods used to estimate user similarity were: ': e,# letter here,
    'There was an issue with using the correlation coefficient.  What was it?': h,# letter here
}

t.test_recs(sol_dict)

"That's right! All of your solutions look good!"

Additionally, let's take a closer look at some of the results.  There are two solution files that you read in to check your results, and you created these objects

* **df_dists** - a dataframe of user1, user2, euclidean distance between the two users
* **all_recs_sol** - a dictionary of all recommendations (key = user, value = list of recommendations)  

`9.` Use these two objects along with the cells below to correctly fill in the dictionary below and complete this notebook!

In [85]:
a = 567
b = 1503
c = 1319
d = 1325
e = 2526710
f = 0
g = 'Use another method to make recommendations - content based, knowledge based, or model based collaborative filtering'

sol_dict2 = {
    'For how many pairs of users were we not able to obtain a measure of similarity using correlation?':e, # letter here,
    'For how many pairs of users were we not able to obtain a measure of similarity using euclidean distance?':f, # letter here,
    'For how many users were we unable to make any recommendations for using collaborative filtering?':c, # letter here,
    'For how many users were we unable to make 10 recommendations for using collaborative filtering?':d, # letter here,
    'What might be a way for us to get 10 recommendations for every user?':g, # letter here   
}

t.test_recs2(sol_dict2)

"That's right! All of your solutions look good!"

In [ ]:
# Use the cells below for any work you need to do!

In [87]:
# Users without recs


In [ ]:
# NaN euclidean distance values


In [ ]:
# Users with less than 10 recs
